In [2]:
NETUID=76
NETWORK="test"
WALLET_NAME="coldkey"
HOTKEY_NAME="hotkey"
TASK_API_URL=""

In [3]:
import requests
import bittensor as bt 
from rich import print as rprint
from typing import Optional,List
from bitagent.schemas.conversation import Conversation
from bitagent.schemas.tool import Tool
subnet = bt.metagraph(netuid=NETUID, network=NETWORK)

# Wallet and validator setup
vali_wallet =  bt.wallet(name=WALLET_NAME, hotkey=HOTKEY_NAME)
vali_dendrite = bt.dendrite(wallet=vali_wallet)

# the request protocol
class QnATask(bt.Synapse):
    urls: List[str] = []   # not used right now - when enabled would allow users to pass in URLs for content
    datas: List[dict] = [] # used to pass in relevant context, could be a company knowledge base or a set of wikipedia pages
    tools: List[dict] = [] # used to pass in tools to be leveraged in answering user query
    prompt: str = ""       # the query / prompt
    response: Optional[dict] = {}
    timeout: Optional[float] = 3.0
    miner_uids: Optional[List[int]] = [] # put our TOP miner into the network as the miner to query (if empty list, a random list of miners will be selected)
    notes = ""
    message_history: Conversation = []
    
    
    def toJSON(self):
        return {"prompt": self.prompt, 
                "urls": self.urls, 
                "datas": self.datas, 
                "tools": self.tools,
                "response": self.response,
                "notes": self.notes,
                "message_history": self.message_history,
                "miner_uids": self.miner_uids,
                "dendrite_process_time": self.dendrite.process_time,
                "dendrite_status_code": self.dendrite.status_code,
                "axon_status_code": self.axon.status_code,}

qna_task = ("generated_qna",1)
pet_tricks_task = ("generated_logic_qna",6)
api_selection_task = ("generated_logic_qna",8)
summarization_task = ("summarization",1)
tool_call_task = ("tool_call",1)
tool_gen_task = ("tool_gen",1)
convo_task = ("conversation",1)
filter_task = ("unfilter",1)

def get_top_miner_uid(subnet):
    return subnet.I.argmax()

def get_task(task_id, sub_task_id):
    task_json = requests.post(f'{TASK_API_URL}/get_new_task', json={"task_name": task_id, "sub_task_id": sub_task_id}).json()
    task_json = task_json['task']
    task = QnATask(prompt=task_json['prompt'], message_history=Conversation.from_list(task_json['message_history']), tools=[Tool(**tool) for tool in task_json['tools']], datas=task_json['datas'], notes=task_json['notes']) # TODO , tools=task_json['task']['tools'])
    return task, task_json["task_id"]

def get_eval(task_id, response):
    return requests.post(f'{TASK_API_URL}/evaluate_task_response', json={"task_id": task_id, "response": response}).json()

In [4]:

def forward(subnet, vali_dendrite, top_miner_uids, task):
    responses = vali_dendrite.query(
        axons=[subnet.axons[uid] for uid in top_miner_uids],
        synapse=task,
        deserialize=False,
        timeout=5*task.timeout,
    )
    return responses

In [20]:
responses = []
tasks = []
for task_type in [qna_task, pet_tricks_task, api_selection_task, summarization_task, tool_seq_selection_task, tool_gen_task, tool_call_task, convo_task, filter_task]:  # going through each task type
    task, task_id = get_task(*task_type)
    tasks.append(task)
    results = forward(subnet, vali_dendrite, [get_top_miner_uid(subnet)], task)

    for result in results:
        try:
            resp = result.response['response']
            responses.append(resp)
        except Exception as e:
            print(e)


In [21]:
# QnA (corpus) Task 
qna_task_synapse = tasks[0]
qna_task_res = responses[0]

print("prompt: ", qna_task_synapse.prompt)
print(qna_task_synapse.message_history)
print(f'Tools: {qna_task_synapse.tools}')
print(f'Response: {qna_task_res}')


prompt:   Which two running backs does the New England Patriots have as pass-catching options, and who could potentially be a suitable traditional rusher to fill the void?
messages=[]
Tools: []
Response: The New England Patriots have Dion Lewis and James White as pass-catching options. LeGarrette Blount could potentially be a suitable traditional rusher to fill the void.


In [22]:
# Pet tricks task
pet_task_synapse = tasks[1]
pet_task_res = responses[1]

print("prompt: ", pet_task_synapse.prompt)
print(pet_task_synapse.message_history)
print(f'Tools: {pet_task_synapse.tools}')
print(f'Response: {pet_task_res}')

prompt:  Given the following Trick Descriptions with numerical IDs:
            1 - 'Stay' - The dog remains in a specific position, such as sitting or lying down, until released by the handler. This trick is essential for safety and control, as it helps prevent the dog from running off or getting into dangerous situations.
2 - 'Wave' - The dog lifts its paw and moves it in a waving motion, typically as a gesture of saying hello or goodbye. This trick requires the dog to understand and perform a specific physical gesture on command, demonstrating its ability to engage in complex social behaviors.
3 - 'Heel' - The dog walks closely beside the handler's leg, maintaining pace and position regardless of the handler's movements. This advanced obedience command is crucial for safe and controlled walking in public spaces, showcasing the dog's discipline and focus on the handler amidst distractions.
4 - 'Lie Down' - The dog moves from a standing or sitting position to lying flat on its belly w

In [23]:
# api selection
api_task_synapse = tasks[2]
api_task_res = responses[2]

print("prompt: ", api_task_synapse.prompt)
print(api_task_synapse.message_history)
print(f'Tools: {api_task_synapse.tools}')
print(f'Response: {api_task_res}')

prompt:  Given the following Tool Descriptions with numerical IDs:
            ['1 - API to read, write, and format Google Sheets data', '2 - Free geo ip information, no registration required. 15k/hour rate limit', '3 - VirusTotal File/URL Analysis', '4 - User management and authentication', '5 - Free JSON storage for small projects']

            And given this unique and purposefully ambiguous tool name: 
            ' Cryptic Cells'

            Which Tool ID (provide numerical number only) is being requested? 
            Tool ID: 
messages=[]
Tools: []
Response: 1


In [24]:
# Summarization
task_synapse = tasks[3]
task_res = responses[3]

print("prompt: ", task_synapse.prompt)
print(task_synapse.message_history)
print(f'Tools: {task_synapse.tools}')
print(f'Response: {task_res}')

prompt:  
messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content='Okay, thats fine. I heard the food at https://www.pizzafactory.store is amazing. Could you analyze this website and tell me what ingredients they offer for their pizzas?'), ChatMessage(role=<ChatRole.ASSISTANT: 'assistant'>, content='Certainly! Heres the list of ingredients available on the Pizza Factory website: \n\n[\n    "Cheese",\n    "Tomatoes",\n    "Mushrooms",\n    "Onions",\n    "Peppers",\n    "Sausage",\n    "Ham",\n    "Salami",\n    "Olives",\n    "Pineapple",\n    "Beef",\n    "Garlic",\n    "Basil",\n    "Artichokes"\n]')]
Tools: [{'name': 'analyze_website', 'description': 'Analyze the performance and content of a website', 'arguments': {'url': {'required': True, 'type': 'string', 'description': 'The URL of the website to analyze'}, 'features': {'required': True, 'type': 'array', 'description': 'The features to analyze on the website'}}}]
Response: [{"role": "tool call", "content": {"name": "analyze_w

In [26]:
# Tool Generation
task_synapse = tasks[4]
task_res = responses[4]

print("prompt: ", task_synapse.prompt)
print(task_synapse.message_history)
print(f'Tools: {task_synapse.tools}')
print(f'Response: {task_res}')

prompt:   "Seeking some exciting action films to view, do you mind assisting me?"
messages=[]
Tools: []
Response: {"name": "get_action_movies", "arguments": {"movie_genre": {"required": true, "type": "string", "description": "The movie genre you want to search for. e.g. 'action', 'comedy', etc."}, "movie_mood": {"required": false, "type": "string", "description": "The mood of the movie you want. e.g. 'exciting', 'heartwarming', etc."}, "year": {"required": false, "type": "integer", "description": "The year of release, can be partial, e.g. 2020s for last decade."}}, "description": "A function to help you find movies according to your preferences. It will return a list of movies based on the given genre, and if provided, the mood and release year."}


In [33]:
# Tool Calling
task_synapse = tasks[5]
task_res = responses[5]

print("prompt: ", task_synapse.prompt)
print(task_synapse.message_history)
print(f'Tools: {task_synapse.tools}')
print(f'Response: {task_res}')

prompt:  
messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content='Ok, then can you tell me how long it might take to deliver this package from New York to Seattle via ground shipping? Do you have access to that information?'), ChatMessage(role=<ChatRole.ASSISTANT: 'assistant'>, content='Sure, I can check the estimated delivery time for you.')]
Tools: [{'name': 'calculate_shipping_cost', 'description': 'Calculate the shipping cost for a package', 'arguments': {'package_details': {'required': True, 'type': 'object', 'description': ''}, 'origin': {'required': True, 'type': 'string', 'description': 'The origin address of the package'}, 'destination': {'required': True, 'type': 'string', 'description': 'The destination address of the package'}}}]
Response: [{"role": "tool call", "content": {"name": "calculate_shipping_cost", "arguments": {"package_details": {}, "origin": "New York", "destination": "Seattle"}}}, {"role": "assistant", "content": "I can use the 'calculate_shipping_cost' t

In [34]:
# Conversation Task
task_synapse = tasks[6]
task_res = responses[6]

print("prompt: ", task_synapse.prompt)
print(task_synapse.message_history)
print(f'Tools: {task_synapse.tools}')
print(f'Response: {task_res}')

prompt:  
messages=[ChatMessage(role=<ChatRole.USER: 'user'>, content=' What is the best approach to employ MATLAB in generating an exclusive, intricate motto for our upgraded artificial intelligence-driven virtual assistant? The slogan must encapsulate the core of the product and pose a difficulty for AI systems to interpret. Can you offer code that leverages machine learning techniques to produce a myriad of slogans, subsequently choosing the most fitting one based on pre-established standards?')]
Tools: []
Response: To employ MATLAB in generating an exclusive, intricate motto for an upgraded artificial intelligence-driven virtual assistant, you can follow these steps:

1. Define the goal: The goal is to generate a motto that represents the core of the product and is difficult for AI systems to interpret.

2. Prepare the data set: Collect a list of phrases or words that represent the core of the product and are challenging for AI systems to interpret.

3. Create a customized algorith

In [35]:
# Unfilter Task
task_synapse = tasks[7]
task_res = responses[7]

print("prompt: ", task_synapse.prompt)
print(task_synapse.message_history)
print(f'Tools: {task_synapse.tools}')
print(f'Response: {task_res}')

prompt:   What made Steve Jobs appear so difficult?


messages=[]
Tools: []
Response: Some say Steve Jobs' difficult reputation stemmed from his perfectionism, which could manifest as impatience, intensity, and a short temper, especially later in life. He was known for his unwavering vision and unwavering, uncompromising stance on design simplicity. 

His intensity could be off-putting to some, but his vision and drive are credited for the immense success of Apple Inc. and the legacy he left.
